### [IMPORTANT] remettre les hashtags (ne pas appliquer de regex dessus)
* rajouter des stop words
* comment mixer les features tf-idf et les autres, pour leur donner du poids? Lire des articles,
* faire deux modèles et mixer
* comment gérer l'ironie?
* Faire une fonction qui va uniquement prédire sur X derniers tweets de la collection tweet

In [1]:
import pymongo as pym
import nltk.data
import re
import string
import unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import TreebankWordTokenizer
import stop_words
from nltk.stem import *
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from scipy.sparse import hstack
from sklearn.linear_model import LogisticRegression
import pickle
from sklearn.model_selection import GridSearchCV

In [2]:
stops = set(['rt','ds','qd','ss','ns','vs','nn','amp','gt','gd','gds','tt','pr','ac','mm', 'qu',
            '``', 'ni', 'ca', 'le', 'les', ' ', 'si', '$', '^', 'via', 'ils'] +
            list('@ن%£€‘:&;') + list('abcdefghijklmnopqrstuvwxyz'))

In [27]:
def tweetPreprocessing(collection, nLastTweets=0, retweet=False):

    tweets = collection.find(filter={'text':{'$exists':True}}, 
                             projection={'_id':False}).sort('$natural',-1).limit(nLastTweets)  
    df = pd.DataFrame()
    listTweets, listCandidats, listSentiments = [], [], []

    for t in tweets: 
        if not retweet: # filtrage des retweets
            if 'rt @' in t['text']:
                continue
        # comptes
        a = t['text'].count('!')
        b = t['text'].count('?')
        c = t['text'].count('#')
        d = t['text'].count('"')
        e = t['text'].count('http')
        
        # mot tronqué
        t['text'] = re.sub(r'\w*…', '', t['text'])
        
        # caracteres speciaux
        t['text'] = re.sub(r'\xad', '-',
                           re.sub(r'\n', ' ', # retrait des sauts de ligne
                                  re.sub(r'\W*(?!\S)', '', # retrait de ce qui n'est pas un mot
                                         re.sub(r'(?:htt)\S*', '', # retrait des liens http
                                                re.sub(r'^rt.*: ', '', # retrait de la mention retweet
                                                       re.sub(r'\d', '', # retrait des chiffres
                                                              re.sub(r',;!?\/\*(){}«»', ' ', t['text'])))))))
        
        t['text'] = re.sub('|'.join(['’', '_', '/', '-', '\'', '“', '\.']), ' ', t['text'])
        
        # accents (il faut laisser ce bloc)
#         t['text'] = re.sub('|'.join('Ééèêë'), 'e', t['text'])
#         t['text'] = re.sub('|'.join('àâä'), 'a', t['text'])
#         t['text'] = re.sub('|'.join('ç'), 'c', t['text'])
#         t['text'] = re.sub('|'.join('œ'), 'oe', t['text'])
#         t['text'] = re.sub('|'.join('Ôôö'), 'o', t['text'])
#         t['text'] = re.sub('|'.join('îï'), 'i', t['text'])
#         t['text'] = re.sub('|'.join('ùû'), 'u', t['text'])
        
        # apostrophes
        t['text'] = re.sub('|'.join([elem + '\'' for elem in 'cdjlmnst']), '', t['text'])
        
        tokenizer = TreebankWordTokenizer()
        t['text'] = tokenizer.tokenize(t['text'])
        t['text'] = [token for token in t['text'] if (token not in stops) and (len(token)>2)]

        while '' in t['text']:
            t['text'].pop('')
            
        if t['text']: # test si liste non vide
            listTweets.append(t['text'])
            try:
                listCandidats.append(t['candidat'])
            except:
                listCandidats.append(None)

            try:
                listSentiments.append(t['sentiment'])
            except:
                listSentiments.append(None)
                
            rec = pd.DataFrame([[a, b, c, d, e]], columns=['!', '?', '#', '"', '_http_'])
            df = df.append(rec, ignore_index=True)
        
    df['text'], df['candidat'], df['sentiment'] = listTweets, listCandidats, listSentiments
    return df

def build_feat_mat(df_tweets):
    vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', decode_error='strict',
                                use_idf=True, norm='l2', binary=False, min_df=.0005, max_df=1.)
    X = vectorizer.fit_transform(df_tweets['text'].apply(' '.join))
    hstack((X, df_tweets[['!', '?', '#', '"', '_http_']]))

    return X

def getSentiments(client, n_predict, retweet, full_retrain=True) : 
    if full_retrain :
        # bases utilisées
        df = tweetPreprocessing(client.tweet.train, 0, retweet)
        df2 = tweetPreprocessing(client.tweet.labelised, 0, retweet)
        try:
            df = pd.concat([df, df2], axis=0, ignore_index=True)
        except:
            pass
        df = df.sample(frac=1.0, replace=False) # mélange des lignes
        X = build_feat_mat(df)
        y = df['sentiment']
        n_samples, vocabulaire = X.shape
        print('Tweets : ' + str(n_samples) + ' / ' + 'Mots : ' + str(vocabulaire))
        params = {'penalty':['l2'], 'C' : np.logspace(-1,1,100),
                  'class_weight':[{-1.0:0.80, 0.0:0.05, 1.0:0.25},'balanced'], 
                  'solver':['lbfgs'], 'multi_class':['ovr','multinomial']}
        lr = LogisticRegression()
        GCV = GridSearchCV(lr, params, verbose=1, n_jobs=-1)
        GCV.fit(X[:-n_predict], y[:-n_predict])
        print(GCV.best_params_)
        model = LogisticRegression(penalty=GCV.best_params_['penalty'],
                                   C=GCV.best_params_['C'], 
                                   class_weight=GCV.best_params_['class_weight'],
                                   solver=GCV.best_params_['solver'], multi_class=GCV.best_params_['multi_class'])   
    
    else :
        f = open('sentiment_model.pkl','rb')
        model = pickle.load(f)
        f.close()
        # bases utilisées
        df = tweetPreprocessing(client.tweet.train, 500, retweet)
        df2 = tweetPreprocessing(client.tweet.labelised, 500, retweet)
        try:
            df = pd.concat([df, df2], axis=0, ignore_index=True)
        except:
            pass
        df = df.sample(frac=1.0, replace=False) # mélange des lignes
        X = build_feat_mat(df)
        y = df['sentiment']
        n_samples, vocabulaire = X.shape
        print('Tweets : ' + str(n_samples) + ' / ' + 'Mots : ' + str(vocabulaire))

    model.fit(X[:-n_predict], y[:-n_predict])
    predictions = model.predict(X[n_samples - n_predict:])
    print('Score', np.sum(predictions == y[n_samples - n_predict:]) / len(predictions)) 
    f = open('sentiment_model.pkl','wb')
    pickle.dump(model, f)
    f.close()
    print('Model saved')
    return predictions

def cleanTrainDb(collection):
    textCleanPipeline = [{"$group":{"_id":"$text", "dups":{"$push":"$_id"},"count":{"$sum":1}}},{"$match":{"count":{"$gt":1}}}]
    duplicates = []
    count = 0
    try:
        for doc in collection.aggregate(textCleanPipeline) :
            it = iter(doc['dups'])
            next(it)
            for id in it:
                count += 1
                duplicates.append(pym.DeleteOne({'_id':id}))
        if duplicates:
            collection.bulk_write(duplicates)
    except:
        pass
    
    print(count, 'doublons retirés.')
    client.close()

In [17]:
#client = pym.MongoClient('localhost', 27018)
client = pym.MongoClient('localhost', 27017)
collection = client.tweet.train
collection2 = client.tweet.labelised
print(collection.count())
print(collection2.count())

4424
1421


### Test du nettoyage des tweets

In [28]:
a = tweetPreprocessing(collection, retweet=True)

In [29]:
for a in a['text'][:20]:
    print(a)

['villumsen', 'jfchauffete', 'macron', 'refuse', 'emission', 'politique', 'tour', 'refuserait', 'serait', 'élu', 'sans', 'jamais', 'avoir', 'été', 'contredit']
['rÉaction', 'bernard', 'lugan', 'aux', 'propos', 'tenus', 'par', 'macron']
['appel', 'rue', 'fillon', 'rêve', 'gaulle', 'mais', 'chienlit', 'est', 'lui', 'lobs']
['lui', 'entre', 'mariage', 'pour', 'tous', 'fillon', 'soupçonne', 'une', 'légère', 'schizophrénie', 'dans', 'son', 'engagement', 'chez']
['mes', 'pensées', 'vont', 'faire', 'ces', 'choses', 'appelle', 'tolerance', 'instruction', 'rip']
['mais', 'pas', 'pareil', 'est', 'gauche', 'donc', 'parfaitement', 'licite']
['ydekerdrel', 'oui', 'mais', 'votre', 'soutien', 'fillon', 'démontre', 'que', 'vos', 'valeurs', 'sont', 'néant', 'assez', 'fourberies']
['brunolemaire', 'droite', 'perdra', 'aussi', 'avec', 'fillon', 'perdra', 'son', 'honneur', 'devra', 'encore', 'trouver', 'nouveau', 'nom']
['fillon', 'trocadéro', 'personnes', 'quand', 'bruno', 'retailleau', 'emballe']
['ump'

### Test du nettoyage doublons

In [17]:
cleanTrainDb(collection)
cleanTrainDb(collection2)

0 doublons retirés.
0 doublons retirés.


## Train et test du modèle

In [50]:
a = getSentiments(client, 1000, retweet=True, full_retrain=True)
# attention en cas de retrain périodique on ne prend que les 500 derniers tweets, il faut donc prédire sur moins de 500
print(len(a[a==1]), len(a[a==0]))

Tweets : 5442 / Mots : 2718
Fitting 3 folds for each of 400 candidates, totalling 1200 fits


[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done 716 tasks      | elapsed:   37.7s
[Parallel(n_jobs=-1)]: Done 966 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 1200 out of 1200 | elapsed:  1.6min finished


{'penalty': 'l2', 'solver': 'lbfgs', 'class_weight': 'balanced', 'C': 0.44306214575838809, 'multi_class': 'ovr'}
Score 0.627
Model saved
330 242


In [8]:
# calculer des f-score, sentiwordnet, bigrammes...